In [ ]:
%matplotlib inline

# DeepFake 數據準備

![deepfake](../Images/deepfaketheme.jpg)

In [9]:
import PIL
from PIL import Image
import dlib
import cv2
import imutils
from imutils.face_utils import *

import glob
import random
import numpy as np

In [12]:
#宣告臉部偵測器，以及載入預訓練的臉部特徵點模型
detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')


In [ ]:
cap = cv2.VideoCapture('videos/jolin.mp4')
i=0
while (cap.isOpened()):#影片轉為每禎圖片
    ret, frame = cap.read()
    if ret == True:
        cv2.imwrite('images data/jolin frames/frame_{0:0>4d}.jpg'.format(i),frame)
    else:
        break
    i+=1
    if i%1000==0:
        print('已處理{0}張'.format(i))
print('共計處理{0}張'.format(i))

In [10]:
cap = cv2.VideoCapture('videos/president2.mp4')
i=1300
while (cap.isOpened()):#影片轉為每禎圖片
    ret, frame = cap.read()
    if ret == True:
        cv2.imwrite('images data/president frames/frame_{0:0>4d}.jpg'.format(i),frame)
    else:
        break
    i+=1
    if i%1000==0:
        print('已處理{0}張'.format(i))
print('共計處理{0}張'.format(i))

已處理2000張
已處理3000張
共計處理3915張


In [11]:
imgs=glob.glob('images data/president frames/frame_*.jpg')
n=0
for im_path in imgs:
    img = cv2.imread(im_path)
    #產生臉部識別
    face_rects = detector(img, 1)
    for i, d in enumerate(face_rects):
        #讀取框左上右下座標
        x1 = d.left()
        y1 = d.top()
        x2 = d.right()
        y2 = d.bottom()
        
        #在臉部位置打框
        crop_img = img[y1:y2, x1:x2, :]
        #要顯示圖形前，需要將BGR轉RGB
        cv2.imwrite(im_path.replace('president frames','president faces'),crop_img)
        n+=1
print('共計擷取出{0}張臉'.format(n))

共計擷取出3217張臉


In [ ]:
imgs=glob.glob('images data/jolin frames/frame_*.jpg')
i=4393
for im_path in imgs[4393:]:
    img = cv2.imread(im_path)
    #產生臉部識別
    face_rects = detector(img, 1)
    for i, d in enumerate(face_rects):
        #讀取框左上右下座標
        x1 = d.left()
        y1 = d.top()
        x2 = d.right()
        y2 = d.bottom()
        
        #在臉部位置打框
        crop_img = img[y1:y2, x1:x2, :]
        #要顯示圖形前，需要將BGR轉RGB
        cv2.imwrite(im_path.replace('jolin frames','jolin faces'),crop_img)
        i+=1
print('共計擷取出{0}張臉'.format(i))

In [13]:
FACE_POINTS = range(17, 68) #臉的輪廓
MOUTH_POINTS = range(48, 68) #嘴
RIGHT_BROW_POINTS =range(17, 22) #右眉毛
LEFT_BROW_POINTS = range(22, 27) #左眉毛
RIGHT_EYE_POINTS = range(36, 42) #右眼
LEFT_EYE_POINTS =range(42, 48) #左眼
NOSE_POINTS = range(27, 35) #鼻子
#JAW_POINTS = range(0, 17) #下巴
ALIGN_POINTS = [LEFT_BROW_POINTS , RIGHT_EYE_POINTS ,LEFT_EYE_POINTS,RIGHT_BROW_POINTS ,NOSE_POINTS , MOUTH_POINTS]

![deepfake](../Images/landmark.jpg)

In [14]:
def Get_landmarks_2D(image):
    w=image.shape[1]
    h=image.shape[0]
    padding=5
    scale=(128-2*padding)/max([image.shape[0],image.shape[1]])

    mean_face_x_ratio = np.array([
        0.000213256, 0.0752622, 0.18113, 0.29077, 0.393397, 0.586856, 0.689483, 0.799124,
        0.904991, 0.98004, 0.490127, 0.490127, 0.490127, 0.490127, 0.36688, 0.426036,
        0.490127, 0.554217, 0.613373, 0.121737, 0.187122, 0.265825, 0.334606, 0.260918,
        0.182743, 0.645647, 0.714428, 0.793132, 0.858516, 0.79751, 0.719335, 0.254149,
        0.340985, 0.428858, 0.490127, 0.551395, 0.639268, 0.726104, 0.642159, 0.556721,
        0.490127, 0.423532, 0.338094, 0.290379, 0.428096, 0.490127, 0.552157, 0.689874,
        0.553364, 0.490127, 0.42689])

    mean_face_y_ratio = np.array([
        0.106454, 0.038915, 0.0187482, 0.0344891, 0.0773906, 0.0773906, 0.0344891,
        0.0187482, 0.038915, 0.106454, 0.203352, 0.307009, 0.409805, 0.515625, 0.587326,
        0.609345, 0.628106, 0.609345, 0.587326, 0.216423, 0.178758, 0.179852, 0.231733,
        0.245099, 0.244077, 0.231733, 0.179852, 0.178758, 0.216423, 0.244077, 0.245099,
        0.780233, 0.745405, 0.727388, 0.742578, 0.727388, 0.745405, 0.780233, 0.864805,
        0.902192, 0.909281, 0.902192, 0.864805, 0.784792, 0.778746, 0.785343, 0.778746,
        0.784792, 0.824182, 0.831803, 0.824182])

    mean_face_x=mean_face_x_ratio*w*scale+padding
    mean_face_y = mean_face_y_ratio*h*scale+padding
    landmarks_2D = np.stack([mean_face_x, mean_face_y], axis=1)
    return landmarks_2D

In [15]:
def umeyama(src, dst, estimate_scale):
    """估計N維相似性轉換參數(可處理有無縮放之狀況) 
    ----------
    src : (M, N) array
        來源點座標
    dst : (M, N) array
        參考點座標
    estimate_scale : bool
        是否估計縮放因子
    Returns
    -------
    T : (N + 1, N + 1)
        The homogeneous similarity transformation matrix. The matrix contains
        NaN values only if the problem is not well-conditioned.
    
    參考文獻
    ----------
    .. [1] "Least-squares estimation of transformation parameters between two
            point patterns", Shinji Umeyama, PAMI 1991, DOI: 10.1109/34.88573
    """

    num = src.shape[0]
    dim = src.shape[1]

    # 計算src 以及 dst的均值
    src_mean = src.mean(axis=0)
    dst_mean = dst.mean(axis=0)

    #將 src 與 dst 減去均值
    src_demean = src - src_mean
    dst_demean = dst  - dst_mean

    # Eq. (38).
    A = np.dot(dst_demean.T, src_demean) / num

    # Eq. (39).
    d = np.ones((dim,), dtype=np.double)
    if np.linalg.det(A) < 0:
        d[dim - 1] = -1

    T = np.eye(dim + 1, dtype=np.double)

    U, S, V = np.linalg.svd(A)

    # Eq. (40) and (43).
    rank = np.linalg.matrix_rank(A)
    if rank == 0:
        return np.nan * T
    elif rank == dim - 1:
        if np.linalg.det(U) * np.linalg.det(V) > 0:
            T[:dim, :dim] = np.dot(U, V)
        else:
            s = d[dim - 1]
            d[dim - 1] = -1
            T[:dim, :dim] = np.dot(U, np.dot(np.diag(d), V))
            d[dim - 1] = s
    else:
        T[:dim, :dim] = np.dot(U, np.dot(np.diag(d), V.T))

    if estimate_scale:
        # Eq. (41) and (42).
        scale = 1.0 / src_demean.var(axis=0).sum() * np.dot(S, d)
    else:
        scale = 1.0

    T[:dim, dim] = dst_mean - scale * np.dot(T[:dim, :dim], src_mean.T)
    T[:dim, :dim] *= scale

    return T

![deepfake](../Images/face_preprocessed.jpg)

In [16]:
def get_image_mask(image,landmarks):
    image_mask=np.zeros(image.shape,dtype=float)
    for points in ALIGN_POINTS:
        try:
            
            hull =cv2.convexHull( np.array(landmarks[points,:] ).reshape((-1,2)).astype(int) ).flatten().reshape( (-1,2) )
            cv2.fillConvexPoly(image_mask,hull,(255,255,255) )
        except Exception as e:
            print(e)
    return image_mask
    
def get_align_face(image,mask,landmarks,size):
    landmarks_2D=Get_landmarks_2D(image)
    
    src_tmp = np.asarray([(int(xy[0]), int(xy[1])) for xy in landmarks[17:]])
    tar_tmp = np.asarray([(int(xy[0]), int(xy[1])) for xy in landmarks_2D])

    M = umeyama(src_tmp,tar_tmp, True)[:2,:]
    align_face = cv2.warpAffine(image, M, (size[1],size[0]), borderMode=cv2.BORDER_REPLICATE)
    align_mask = cv2.warpAffine(mask, M, (size[1], size[0]), borderMode=cv2.BORDER_REPLICATE)
    return align_face,align_mask

![deepfake](../Images/warpAffine.png)

In [7]:
imgs=glob.glob('images data/jolin faces/frame_*.jpg')
i=0
for im_path in imgs:
    faceimg=cv2.imread(im_path)
    d=dlib.rectangle(0, 0,faceimg.shape[0], faceimg.shape[1])
    shape=landmark_predictor(faceimg, d)
    landmark = shape_to_np(shape)
    
    try:
        image_mask=get_image_mask(faceimg,landmark)
        cv2.imwrite(im_path.replace('jolin faces','jolin masks'),image_mask)

        align_face,align_mask=get_align_face(faceimg,image_mask,landmark,(128,128))
        cv2.imwrite(im_path.replace('jolin faces','jolin align faces'),align_face)
        cv2.imwrite(im_path.replace('jolin faces','jolin align masks'),align_mask)
    except Exception as e:
        print(e)
    i+=1
    if i%200==0:
        print('已處理{0}張'.format(i))
print('共計擷取出{0}張臉'.format(i))

已處理200張
已處理400張
已處理600張
已處理800張
已處理1000張
已處理1200張
已處理1400張
已處理1600張
已處理1800張
已處理2000張
已處理2200張
已處理2400張
已處理2600張
已處理2800張
已處理3000張
已處理3200張
已處理3400張
已處理3600張
已處理3800張
已處理4000張
已處理4200張
已處理4400張
已處理4600張
已處理4800張
已處理5000張
共計擷取出5057張臉


In [17]:
imgs=glob.glob('images data/president faces/frame_*.jpg')
i=0
for im_path in imgs:
    faceimg=cv2.imread(im_path)
    d=dlib.rectangle(0, 0,faceimg.shape[0], faceimg.shape[1])
    shape=landmark_predictor(faceimg, d)
    landmark = shape_to_np(shape)
    
    try:
        image_mask=get_image_mask(faceimg,landmark)
        cv2.imwrite(im_path.replace('president faces','president masks'),image_mask)

        align_face,align_mask=get_align_face(faceimg,image_mask,landmark,(128,128))
        cv2.imwrite(im_path.replace('president faces','president align faces'),align_face)
        cv2.imwrite(im_path.replace('president faces','president align masks'),align_mask)
    except Exception as e:
        print(e)
    i+=1
    if i%200==0:
        print('已處理{0}張'.format(i))
print('共計擷取出{0}張臉'.format(i))

已處理200張
已處理400張
已處理600張
已處理800張
已處理1000張
已處理1200張
已處理1400張
已處理1600張
已處理1800張
已處理2000張
已處理2200張
已處理2400張
已處理2600張
已處理2800張
已處理3000張
已處理3200張
共計擷取出3203張臉


![deepfake](../Images/deepfake-processed.jpg)